## About the notebook
Here I have collected most recent depressive tweets from the twitter, I used selenium for scrapping the twitter website and scrapped 2307 tweets which will be preprocessed and merged with sentiment 140 to create a final data set.

### The reason I used selenium is
##### 1. Using api or any other tool has limitation as we had to generate token for that and still we can only fetch relatively less amount of tweets.
##### 2. I wanted to learn about web scraping and I believe that I had this great opportunity to learn and at same time use scrapping for my mini project.

# 1. Setting up Environment for Gathering Data
## Selenium Automation on Google Colab Notebook
The code below is the solution of unexpected excitation of chrome driver.
Source:
https://github.com/googlecolab/colabtools/issues/3347#issuecomment-1387453484


In [2]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.DiRF2wZiSx/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Lp2HLZMePm/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.0DQU8kGCr8/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

In [3]:
import pandas as pd
import re
import csv

# 2. Gathering Depressive Tweets
We want to collect information from Twitter using hashtags for depression, such as #depressed, #depression, #loneliness, and #hopelessness.

## Function to scrape data from twitter
The function take keyword and number of tweets to scrape as argument and returns a list of tuple.Each tuple in the list contains information of a particular tweet

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
%cd /content/drive/MyDrive/Depression_Analysis_by_Tweets

/content/drive/MyDrive/Depression_Analysis_by_Tweets


In [38]:

# from getpass import getpass
from time import sleep
# import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
tweet_ids = set()
id=0
field_names=['id','username','tweet','label']
with open('depressive_tweets.csv', 'a') as f_object:
    writer = csv.DictWriter(f_object,delimiter=',', fieldnames=field_names)
    writer.writeheader()

########################################################
def get_tweetcard_data(card):
    """Extract data from tweet"""
    UserTag = card.find_element(By.XPATH,".//span[contains(text(),'@')]").text
    # try:
    #     TimeStamp = card.find_element(By.XPATH,".//time").get_attribute('datetime')
    # except NoSuchElementException:
    #     return
    global id
    Id = id
    id = id + 1
    Tweet = card.find_element(By.XPATH,".//div[@data-testid='tweetText']").text
    # Reply = card.find_element(By.XPATH, ".//div[@data-testid='reply']").text
    # reTweet = card.find_element (By .XPATH, ".//div[@data-testid='retweet']").text
    # Like =card.find_element (By.XPATH,".//div[@data-testid='like']").text
    label=1
    tweet = (Id,UserTag,Tweet,label)
    return tweet

########################################################
def get_tweets(keyword):
    if "#" not in keyword:
        URL="https://twitter.com/search?q="+keyword+"&src=typed_query&f=top"
    else:
        URL="https://twitter.com/search?q=%23"+keyword[1:]+"&src=typed_query&f=top"
    driver.get(URL)
    print(driver.title)
    sleep(3)

    #Get all the tweets in the page
    # tweet_data=[]
    # tweet_ids=set()
    i=0
    last_position = driver.execute_script("return document.body.scrollHeight;")
    scrolling = True

    while scrolling:
        cards = driver.find_elements(By.XPATH,"//article[@data-testid='tweet']")
        for card in cards[:7]:
            data = get_tweetcard_data(card)
            if data:
                tweet_id = ''.join(data[1:3])
                if tweet_id not in tweet_ids:
                  tweet_ids.add(tweet_id)
                  # tweet_data.append(data)
                  with open('depressive_tweets.csv', 'a') as f_object:
                    writer = csv.writer(f_object)
                    writer.writerow(data)
                    i=i+1

        scroll_attempt = 0
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
             # Wait to load page
            sleep(3)
            
            #check scroll position
            curr_position = driver.execute_script("return document.body.scrollHeight;")
            if(last_position==curr_position):
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= 5:
                  scrolling = False
                  break
                else:
                    sleep(5)
            else:
                last_position = curr_position
                break
        if(i>150):
            break
########################################################

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.headless = True
driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

<ipython-input-38-888591cb7c44>:94: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
<ipython-input-38-888591cb7c44>:95: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


## Gathering tweets of hashtags 
["#depression","#depressed","#loneliness", "#hopelessness","#hopeless","#lonely","#suicide"]

In [39]:
depressive_tags = ["#depression","#depressed","#loneliness",
                   "#hopelessness","#hopeless","#lonely",
                   "#suicide","#alone","#anxiety",
                   "#depressionhurts","	#ihavecripplingdepression",
                   "#recovery","#depressionquotes","#depressionhelp",
                   "#depressionisreal","#depressions","#anxietyanddepression",
                   "#bipolardisorder","#bipolar", "#severeanxiety","#bpd",
                   "#Depression","#Depressed","#Loneliness",
                   "#Hopelessness","#Hopeless","#Lonely",
                   "#Suicide","#Alone","#Anxiety",
                   "#Depressionhurts","#Recovery","#Depressionquotes","#Depressionhelp",
                   "#Depressionisreal","#Depressions","#Anxietyanddepression",
                   "#Bipolardisorder","#Bipolar", "#Severeanxiety","#BPD"
                   ]
for depressive_tag in depressive_tags:
  get_tweets(depressive_tag)

Twitter
Twitter

Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter

Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter

Twitter
Twitter

Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter
Twitter


Twitter


# 3. Cleaning Depressive Tweets Dataset
##### Here, we deleted any unnecessary tweets using a variety of techniques. This script will provide a dataset with a narrowed-down selection of tweets that might have a depressing tone. 

##### To avoid the machine learning algorithm being tricked into just using depressed hashtags, the script also removes all hashtags from the tweets. To precisely distinguish between depressive and non-depressive messages in the final dataset, the dataset is then individually examined and labelled.

In [128]:
df = pd.read_csv("depressive_tweets.csv")
len(df)

2505

### 1. remove entries that contain positive, or medical sounding tags

In [129]:
to_remove = ["medical","trearment","mission","support","social","motivation","podcast"]
mask1 = df.tweet.apply(lambda x: any(item for item in to_remove if item in x))
df[mask1].tweet.tail()

2422    It's the constant expectation of people to alw...
2429    Sasha struggled to understand why the treatmen...
2479    Merging #RDOC & Alt Model of Personality to di...
2486    How a #BPD Wife Ruined My Life: Tips and Advic...
2489    Go check out my tiktok yall, been needing a lo...
Name: tweet, dtype: object

In [130]:
df[mask1==False].tweet.head(10)

1     #Depression is when you feel that the whole wo...
2     #depression \n\nMYTH\n“I don't have depression...
4     There is growing evidence #loneliness is assoc...
5     No one who knows me on a personal level, shoul...
6     #Depression can happen to anyone. \n\nIf you h...
7     I finally got a good picture of Ozzy Bozzy Buz...
8     #Depression -\n\n- Feeling worthless, a burden...
9         me to my anxiety \n#anxiety #life #depression
10                   #depression is a very sucky thing.
11         Mý #depression is the biggest pain in my ass
Name: tweet, dtype: object

In [131]:
#applying the mask
df = df[mask1==False]
len (df)

2232

### 2. remove entries with more than three hashtags, as it may be promotional messages

In [132]:
mask2 = df.tweet.apply(lambda x: x.count("#") < 7)
df = df[mask2]
len (df)

1752

### 3. remove entries with less than x chars / words

In [133]:
mask3 = df.tweet.apply(lambda x: len(x) > 10)
df = df[mask3]
len(df)

1739

In [146]:
df.head()

,id,username,tweet,label
1,1,@mysticcshadow,#Depression is when you feel that the whole wo...,1
2,2,@pmalston,#depression \n\nMYTH\n“I don't have depression...,1
4,4,@KulikovUNIATF,There is growing evidence #loneliness is assoc...,1
5,5,@Seun_Ambassador,"No one who knows me on a personal level, shoul...",1
6,6,@WHO_Zimbabwe,#Depression can happen to anyone. \n\nIf you h...,1


In [152]:
df = df.drop(['username'], axis=1)
df.head()

,id,tweet,label
1,1,#Depression is when you feel that the whole wo...,1
2,2,#depression \n\nMYTH\n“I don't have depression...,1
4,4,There is growing evidence #loneliness is assoc...,1
5,5,"No one who knows me on a personal level, shoul...",1
6,6,#Depression can happen to anyone. \n\nIf you h...,1


In [153]:
df.tail()

,id,tweet,label
2498,3813,Borderline personality disorder and the abusiv...,1
2499,3817,"A Life Worth Living. Over the coming weeks, we...",1
2501,3822,Adult Children of Borderline Parents: The Comp...,1
2502,3827,The annoying sleep pattern… still awake at 4 a...,1
2504,3832,I knew exactly what was going to happen. I tri...,1


In [154]:
len(df)

1739

### id is unique but due to connectivity issue while scraping, the ids are not continous

# 3. Fetching sentiment 140 dataset from kaggle


In [134]:
!pip install -q kaggle

In [7]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tusharpuri10","key":"2c7f8280b894d1225617f1fa4a51bef7"}'}

In [135]:
!ls -lha kaggle.json

-rw------- 1 root root 68 Jan 24 12:30 kaggle.json


In [136]:
!mkdir -p ~/.kaggle #Create the directory

In [137]:
!cp kaggle.json ~/.kaggle/ # moving json file in the folder

In [138]:
!chmod 600 /root/.kaggle/kaggle.json # set permissions to the json file

In [139]:
!pwd #Print Working Directory

/content/drive/MyDrive/Depression_Analysis_by_Tweets


In [140]:
!kaggle datasets list

ref                                                             title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ahsan81/hotel-reservations-classification-dataset               Hotel Reservations Dataset                        480KB  2023-01-04 12:50:31           4554        169  1.0              
senapatirajesh/netflix-tv-shows-and-movies                      Latest Netflix TV shows and movies                  1MB  2023-01-14 17:03:12           1200         35  0.88235295       
johnny1994/divorce-rates-data-should-you-get-married            Divorce Rates Data: Should You Get Married?        22KB  2023-01-15 12:49:06            755         30  0.88235295       
rakkesharv/spotify-top-10000-streamed-songs                     Spotif

In [141]:
!kaggle datasets download -d kazanova/sentiment140

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [142]:
!unzip sentiment140.zip

Archive:  sentiment140.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [165]:
corpus = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1')
len(corpus)

1599999

In [167]:
corpus.set_axis(["label", "id", "a","b","c","tweet"], axis=1,inplace=True)

In [168]:
corpus.tail(20)

,label,id,a,b,c,tweet
1599979,4,2193578679,Tue Jun 16 08:38:56 PDT 2009,NO_QUERY,puchal_ek,@myheartandmind jo jen by nemuselo zrovna tÃ© ...
1599980,4,2193578716,Tue Jun 16 08:38:57 PDT 2009,NO_QUERY,youtubelatest,Another Commenting Contest! [;: Yay!!! http:/...
1599981,4,2193578739,Tue Jun 16 08:38:57 PDT 2009,NO_QUERY,Mandi_Davenport,@thrillmesoon i figured out how to see my twee...
1599982,4,2193578758,Tue Jun 16 08:38:57 PDT 2009,NO_QUERY,xoAurixo,"@oxhot theri tomorrow, drinking coffee, talkin..."
1599983,4,2193578847,Tue Jun 16 08:38:57 PDT 2009,NO_QUERY,RobFoxKerr,You heard it here first -- We're having a girl...
1599984,4,2193578982,Tue Jun 16 08:38:58 PDT 2009,NO_QUERY,LISKFEST,"if ur the lead singer in a band, beware fallin..."
1599985,4,2193579087,Tue Jun 16 08:38:58 PDT 2009,NO_QUERY,marhgil,@tarayqueen too much ads on my blog.
1599986,4,2193579092,Tue Jun 16 08:38:58 PDT 2009,NO_QUERY,cathriiin,@La_r_a NEVEER I think that you both will get...
1599987,4,2193579191,Tue Jun 16 08:38:59 PDT 2009,NO_QUERY,tellman,@Roy_Everitt ha- good job. that's right - we g...
1599988,4,2193579211,Tue Jun 16 08:38:59 PDT 2009,NO_QUERY,jazzstixx,@Ms_Hip_Hop im glad ur doing well


The polarity of the tweet in sentiment 140 is (0 = negative, 2 = neutral, 4 = positive)

In [169]:
corpus.label.value_counts()

4    800000
0    799999
Name: label, dtype: int64

## Fetching 9000 positive tweets

In [258]:
df2 = corpus.copy()

In [259]:
df2 = df2.drop(range(0,799999))
df2.label.value_counts()

4    800000
Name: label, dtype: int64

dropping 800000 - 9000 = 791000 tweets

In [260]:
df2 = df2.drop(range(799999,799999+791000))
df2.label.value_counts()

4    9000
Name: label, dtype: int64

In [261]:
df2.head()

,label,id,a,b,c,tweet
1590999,4,2191485827,Tue Jun 16 05:22:32 PDT 2009,NO_QUERY,CaroMcFly,"@xEviLovesMcFLYx haha, that's cool Geiselwind..."
1591000,4,2191485875,Tue Jun 16 05:22:33 PDT 2009,NO_QUERY,LarissaAutobots,@Hatz94 okay I will go take a look
1591001,4,2191485919,Tue Jun 16 05:22:33 PDT 2009,NO_QUERY,WhitWhit140,"Everybody Loves Little Chris nadadadada, yeah..."
1591002,4,2191486003,Tue Jun 16 05:22:34 PDT 2009,NO_QUERY,starbucksgirl02,@NickkkJonasss congrats for your graduation! (...
1591003,4,2191486063,Tue Jun 16 05:22:34 PDT 2009,NO_QUERY,AYellowHouse,@TracyeDukes It certainly is a good day


In [262]:
df2 = df2.drop(['a','b','c'],axis=1)
df2.head()

,label,id,tweet
1590999,4,2191485827,"@xEviLovesMcFLYx haha, that's cool Geiselwind..."
1591000,4,2191485875,@Hatz94 okay I will go take a look
1591001,4,2191485919,"Everybody Loves Little Chris nadadadada, yeah..."
1591002,4,2191486003,@NickkkJonasss congrats for your graduation! (...
1591003,4,2191486063,@TracyeDukes It certainly is a good day


In [263]:
columns_titles = ["id","tweet","label"]
df2=df2.reindex(columns=columns_titles)
df2.head()

,id,tweet,label
1590999,2191485827,"@xEviLovesMcFLYx haha, that's cool Geiselwind...",4
1591000,2191485875,@Hatz94 okay I will go take a look,4
1591001,2191485919,"Everybody Loves Little Chris nadadadada, yeah...",4
1591002,2191486003,@NickkkJonasss congrats for your graduation! (...,4
1591003,2191486063,@TracyeDukes It certainly is a good day,4


##### Changing 4 to 0 as my convention:
##### 1 -> depressive
##### 0 -> cheerful

In [264]:
df2 = df2.replace(4,0)
df2.head()

,id,tweet,label
1590999,2191485827,"@xEviLovesMcFLYx haha, that's cool Geiselwind...",0
1591000,2191485875,@Hatz94 okay I will go take a look,0
1591001,2191485919,"Everybody Loves Little Chris nadadadada, yeah...",0
1591002,2191486003,@NickkkJonasss congrats for your graduation! (...,0
1591003,2191486063,@TracyeDukes It certainly is a good day,0


In [265]:
df2.tail()

,id,tweet,label
1599994,2193601966,Just woke up. Having no school is the best fee...,0
1599995,2193601969,TheWDB.com - Very cool to hear old Walt interv...,0
1599996,2193601991,Are you ready for your MoJo Makeover? Ask me f...,0
1599997,2193602064,Happy 38th Birthday to my boo of alll time!!! ...,0
1599998,2193602129,happy #charitytuesday @theNSPCC @SparksCharity...,0


In [266]:
df_final = pd.concat([df, df2])
df_final = df_final.reset_index(drop=True)

In [267]:
df_final.head()

,id,tweet,label
0,1,#Depression is when you feel that the whole wo...,1
1,2,#depression \n\nMYTH\n“I don't have depression...,1
2,4,There is growing evidence #loneliness is assoc...,1
3,5,"No one who knows me on a personal level, shoul...",1
4,6,#Depression can happen to anyone. \n\nIf you h...,1


In [268]:
df_final.tail()

,id,tweet,label
10734,2193601966,Just woke up. Having no school is the best fee...,0
10735,2193601969,TheWDB.com - Very cool to hear old Walt interv...,0
10736,2193601991,Are you ready for your MoJo Makeover? Ask me f...,0
10737,2193602064,Happy 38th Birthday to my boo of alll time!!! ...,0
10738,2193602129,happy #charitytuesday @theNSPCC @SparksCharity...,0


In [269]:
for ind in df_final.index:
  df_final['id'][ind]=ind+1

<ipython-input-269-cdf24b07b4cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['id'][ind]=ind+1


In [270]:
df_final.head()

,id,tweet,label
0,1,#Depression is when you feel that the whole wo...,1
1,2,#depression \n\nMYTH\n“I don't have depression...,1
2,3,There is growing evidence #loneliness is assoc...,1
3,4,"No one who knows me on a personal level, shoul...",1
4,5,#Depression can happen to anyone. \n\nIf you h...,1


In [271]:
df_final.tail()

,id,tweet,label
10734,10735,Just woke up. Having no school is the best fee...,0
10735,10736,TheWDB.com - Very cool to hear old Walt interv...,0
10736,10737,Are you ready for your MoJo Makeover? Ask me f...,0
10737,10738,Happy 38th Birthday to my boo of alll time!!! ...,0
10738,10739,happy #charitytuesday @theNSPCC @SparksCharity...,0


In [272]:
df_final.to_csv('Dataset.csv')